# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_nontata_promoters"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_nontata_promoters 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2021-11-23 01:27:38.944978: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 01:27:38.944998: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-11-23 01:27:42.026221: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-23 01:27:42.026282: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-11-23 01:27:42.027646: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-11-23 01:27:45.855583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'g', 'c', 't', 'a']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


  1/424 [..............................] - ETA: 8:07 - loss: 0.8225 - binary_accuracy: 0.4219 - f1_score: 0.0000e+00

  5/424 [..............................] - ETA: 6s - loss: 0.6945 - binary_accuracy: 0.5781 - f1_score: 0.0373      

  9/424 [..............................] - ETA: 6s - loss: 0.6660 - binary_accuracy: 0.6198 - f1_score: 0.2640

 13/424 [..............................] - ETA: 6s - loss: 0.6320 - binary_accuracy: 0.6635 - f1_score: 0.4096

...

414/424 [============================>.] - ETA: 0s - loss: 0.2525 - binary_accuracy: 0.8956 - f1_score: 0.8946

417/424 [============================>.] - ETA: 0s - loss: 0.2529 - binary_accuracy: 0.8953 - f1_score: 0.8943

420/424 [============================>.] - ETA: 0s - loss: 0.2532 - binary_accuracy: 0.8952 - f1_score: 0.8942

423/424 [============================>.] - ETA: 0s - loss: 0.2534 - binary_accuracy: 0.8952 - f1_score: 0.8941

424/424 [==============================] - 8s 18ms/step - loss: 0.2536 - binary_accuracy: 0.8952 - f1_score: 0.8940


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/142 [..............................] - ETA: 44s - loss: 0.4231 - binary_accuracy: 0.8125 - f1_score: 0.7143

  8/142 [>.............................] - ETA: 1s - loss: 0.5123 - binary_accuracy: 0.7910 - f1_score: 0.7293 

 13/142 [=>............................] - ETA: 1s - loss: 0.4973 - binary_accuracy: 0.7981 - f1_score: 0.7490

 18/142 [==>...........................] - ETA: 1s - loss: 0.5042 - binary_accuracy: 0.8038 - f1_score: 0.7461

 22/142 [===>..........................] - ETA: 1s - loss: 0.5104 - binary_accuracy: 0.8047 - f1_score: 0.7407

...

131/142 [==========================>...] - ETA: 0s - loss: 0.5111 - binary_accuracy: 0.7965 - f1_score: 0.7349

136/142 [===========================>..] - ETA: 0s - loss: 0.5116 - binary_accuracy: 0.7954 - f1_score: 0.7336

142/142 [==============================] - ETA: 0s - loss: 0.5129 - binary_accuracy: 0.7948 - f1_score: 0.7328

142/142 [==============================] - 2s 11ms/step - loss: 0.5129 - binary_accuracy: 0.7948 - f1_score: 0.7328


[0.5129161477088928, 0.7947753071784973, 0.7328321933746338]